# User Guide

## Class Hierarchy

$\langle {FiniteAlgebra} \rangle \rightarrow$ Magma $\rightarrow$ Semigroup $\rightarrow$ Monoid $\rightarrow$ Group $\rightarrow$ Ring $\rightarrow$ Field

* **Magma** -- a set with a binary operation:  $\langle S, \circ \rangle$, where $S$ is a set and $\circ: S \times S \to S$

* **Semigroup** -- an associative Magma:  for any $a,b,c \in S \Rightarrow a \circ (b \circ c) = (a \circ b) \circ c$

* **Monoid** -- a Semigroup with identity element:  $\exists e \in S$, such that, for all $a \in S, a \circ e = e \circ a = a$

* **Group** -- a Monoid with inverse elements:  $\forall a \in S, \exists a^{-1} \in S$, such that, $a \circ a^{-1} = a^{-1} 
\circ a = e$

* **Ring** -- $\langle S, +, \times \rangle$, where $\langle S, + \rangle$ is a commutative Group, $\langle S, \times \rangle$ is a Semigroup, and $\times$ distributes over $+$

* **Field** -- a Ring $\langle S, +, \times \rangle$, where $\langle S\setminus{\{0\}}, \times \rangle$ is a commutative Group.

## Finite Algebra: Internal Representation

Internally, a ``FiniteAlgebra`` consists of the following quantities:

* **name**: (``str``) A short name for the algebra;
* **description**: (``str``) Any additional, useful information about the algebra;
* **elements**: (``list`` of ``str``) Names of the algebras’s elements.
* **table**: (``list`` of ``list`` of ``int``) The algebra’s multiplication
  table, where each list in the list represents a row of the table, and
  each integer represents the position of an element in ‘element_names’.
  Optionally, element names (``str``) may be used in the table, rather 
  than integers.
* **table2**: (OPTIONAL) Similar to *table*, above. Required when defining a Ring or Field.

**NOTE**: The type of table required here is known as a [Cayley Table](https://en.wikipedia.org/wiki/Cayley_table).  All of the properties of a finite algebra can be derived from its Cayley Table.  For this reason, this module includes a ``CayleyTable`` class for storing the table and methods associated with it.

## Algebra Constuction Examples

In a nutshell, use the function, ``make_finite_algebra`` for all algebra construction.

Although individual algebras (Magma, Semigroup, etc.) have their own individual constructors, requiring the quantities described above, the **recommended** way to construct an algebra is to use the function, ``make_finite_algebra``, using one of the following three approaches to inputs:

1. Enter **individual values** corresponding to the quantities in its Internal Representation, described above.
1. Enter a **Python dictionary** (``dict``), with keys and values corresponding to the individual values, described above.
1. Enter the **path to a JSON file** (``str``) that corresponds to the dictionary, described above.
   
``make_finite_algebra`` uses the table(s) to determine what type of algebra it supports and returns the appropriate algebra.

In the following examples, the only algebra constructor used is ``make_finite_algebra``.

### Group

We'll start in the middle of the hierarchy of algebras, the Group.

Finite algebra elements, here, are always represented as strings; and, although a Cayley table can be entered (and displayed) using strings, they are represented internally (and displayed by default) as 2-dimensional, square arrays of integers that represent the positions of elements in the element list.

In [1]:
>>> from finite_algebras import make_finite_algebra

>>> z3 = make_finite_algebra('Z3',
                             'Cyclic group of order 3',
                             ['e', 'a', 'a^2'],
                             [[ 'e' ,  'a' , 'a^2'],
                              [ 'a' , 'a^2',  'e' ],
                              ['a^2',  'e' ,  'a' ]]
                            )
>>> z3

Group(
'Z3',
'Cyclic group of order 3',
['e', 'a', 'a^2'],
[[0, 1, 2], [1, 2, 0], [2, 0, 1]]
)

Printing an algebra converts it to a string containing its class name, algebra name, and the unique ID of the algebra instance:

In [2]:
>>> print(z3)

<Group:Z3, ID:140321017988368>


The ``about`` prints information about an algebra.  Set ``use_element_names`` to ``True`` to see the Cayley table printed using element names (``str``) rather than element positions (``int``).

In [3]:
>>> z3.about(use_table_names=True)


Group: Z3
Instance ID: 140321017988368
Description: Cyclic group of order 3
Order: 3
Identity: e
Associative? Yes
Commutative? Yes
Elements:
   Index   Name   Inverse  Order
      0       e       e       1
      1       a     a^2       3
      2     a^2       a       3
Cayley Table (showing names):
[['e', 'a', 'a^2'], ['a', 'a^2', 'e'], ['a^2', 'e', 'a']]


### Group Properties

In [4]:
>>> z3.is_associative()  # Only Magmas are non-associative

True

In [5]:
>>> z3.is_commutative()

True

In [6]:
>>> z3.is_abelian()

True

The ``identity`` method (property) returns the algebra's identity element, if it exists.

If the identity doesn't exist, then ``None`` is returned.

In [7]:
>>> z3.identity

'e'

In [8]:
>>> z3.inv('a')  # Get an element's inverse, if it exists

'a^2'

Internal to algebras, tables are stored as instances of the ``CayleyTable`` class:

In [9]:
>>> z3.table

CayleyTable([[0, 1, 2], [1, 2, 0], [2, 0, 1]])

### Binary Operation

In [10]:
>>> z3.op()  # zero arguments returns the identity, if it exists

'e'

If only one argument is given to the binary operation, then that argument is simply returned; unless it is not a valid element of the algebra, in which case an exception is raised.

In [11]:
>>> z3.op('a')

'a'

For $Z_3$, $a \circ a = a^2$

In [12]:
>>> z3.op('a', 'a')

'a^2'

and $a \circ a \circ a = a \circ a^2 = a^2 \circ a = e$.

In [13]:
>>> z3.op('a', 'a', 'a') == z3.op('a', 'a^2') == z3.op('a^2', 'a') == 'e'

True

Note, however, that the function, ``op``, can only be used with elements (``str``) that are members of the element list.  So, since 'a^3' is not a string in the element list, it cannot be used in function ``op``.

In [14]:
>>> try:
>>>     z3.op('a^3')
>>> except Exception as exc:
>>>     print(exc)

a^3 is not a valid element name


### "Subtraction" in Groups

The method, ``sub``, is a convenience method for computing "$x - y$", that is, $x \circ y^{-1}$ where $x, y \in \langle G, \circ \rangle$.

In [15]:
>>> x = 'a'
>>> y = 'a^2'
>>> print(f"For example, \"{x} - {y}\" = {x} * {z3.inv(y)} = {z3.op(x, z3.inv(y))}")

For example, "a - a^2" = a * a = a^2


Or, more succinctly:

In [16]:
>>> z3.sub(x, y)

'a^2'

### Magma

**Magma** -- a set with a binary operation:  $\langle S, \circ \rangle$, where $S$ is a finite set and $\circ: S \times S \to S$

**Rock-Paper-Scissors**

See https://en.wikipedia.org/wiki/Commutative_magma

* $\langle S, \circ \rangle$, where $S = \{r,p,s\}$
* For all $x, y \in S$, if $x$ *beats* $y$, then $x \circ y = y \circ x = x$
* Also, for all $x \in S$, $xx = x$

From the rule in the second bullet, above, this algebra is obviously commutative.

In [17]:
>>> rps = make_finite_algebra('RPS',
                              'Rock, Paper, Scissors Magma',
                              ['r', 'p', 's'],
                              [['r', 'p', 'r'],
                               ['p', 'p', 's'],
                               ['r', 's', 's']])

>>> rps.about()


Magma: RPS
Instance ID: 140321287422352
Description: Rock, Paper, Scissors Magma
Order: 3
Elements: ['r', 'p', 's']
Identity: None
Associative? No
Commutative? Yes
Has Inverses? No
Cayley Table (showing indices):
[[0, 1, 0], [1, 1, 2], [0, 2, 2]]


Paper beats Rock:

In [18]:
>>> rps.op('r', 'p')

'p'

In [19]:
>>> if rps.identity is None:
>>>     print("RPS does not have an identity element")

RPS does not have an identity element


For convenience, the method, ``has_identity``, returns True or False, depending on whether an algebra has an identity.

In [20]:
>>> rps.has_identity()

False

The next section demonstrates that a Magma can have an identity element, as long as the Magma is not associative, otherwise ``make_finite_algebra`` would output a Monoid.

### Magma with Identity Element

In [21]:
>>> mag = make_finite_algebra('Whatever',
                              'Magma with Identity',
                              ['e', 'a', 'b'],
                              [['e', 'a', 'b'],
                               ['a', 'e', 'a'],
                               ['b', 'b', 'a']])

>>> mag.about()


Magma: Whatever
Instance ID: 140321287423248
Description: Magma with Identity
Order: 3
Elements: ['e', 'a', 'b']
Identity: e
Associative? No
Commutative? No
Has Inverses? No
Cayley Table (showing indices):
[[0, 1, 2], [1, 0, 1], [2, 2, 1]]


### Semigroup

**Semigroup** -- an associative Magma:  for any $a,b,c \in S \Rightarrow a \circ (b \circ c) = (a \circ b) \circ c$

Reference: [Groupoids and Smarandache Groupoids](https://arxiv.org/ftp/math/papers/0304/0304490.pdf) by W. B. Vasantha Kandasamy

In [22]:
>>> sg = make_finite_algebra(
    'Example 1.4.1',
    'See: Groupoids and Smarandache Groupoids by W. B. Vasantha Kandasamy',
    ['a', 'b', 'c', 'd', 'e', 'f'],
    [[0, 3, 0, 3, 0, 3],
     [1, 4, 1, 4, 1, 4],
     [2, 5, 2, 5, 2, 5],
     [3, 0, 3, 0, 3, 0],
     [4, 1, 4, 1, 4, 1],
     [5, 2, 5, 2, 5, 2]]
)

>>> sg.about()


Semigroup: Example 1.4.1
Instance ID: 140321287435856
Description: See: Groupoids and Smarandache Groupoids by W. B. Vasantha Kandasamy
Order: 6
Elements: ['a', 'b', 'c', 'd', 'e', 'f']
Identity: None
Associative? Yes
Commutative? No
Has Inverses? No
Cayley Table (showing indices):
[[0, 3, 0, 3, 0, 3],
 [1, 4, 1, 4, 1, 4],
 [2, 5, 2, 5, 2, 5],
 [3, 0, 3, 0, 3, 0],
 [4, 1, 4, 1, 4, 1],
 [5, 2, 5, 2, 5, 2]]


Since the element in the 0,1 position of the table is 3, it follows that, $a \circ b = d$:

In [23]:
>>> sg.op('a', 'b')

'd'

### Monoid

**Monoid** -- a Semigroup with identity element:  $\exists e \in S$, such that, for all $a \in S, a \circ e = e \circ a = a$

In [24]:
>>> m4 = make_finite_algebra('M4',
                             'Example of a commutative monoid',
                             ['a', 'b', 'c', 'd'],
                             [[0, 0, 0, 0],
                              [0, 1, 2, 3],
                              [0, 2, 0, 2],
                              [0, 3, 2, 1]])

>>> m4.about(use_table_names=True)


Monoid: M4
Instance ID: 140320620173904
Description: Example of a commutative monoid
Order: 4
Elements: ['a', 'b', 'c', 'd']
Identity: b
Associative? Yes
Commutative? Yes
Has Inverses? No
Cayley Table (showing names):
[['a', 'a', 'a', 'a'],
 ['a', 'b', 'c', 'd'],
 ['a', 'c', 'a', 'c'],
 ['a', 'd', 'c', 'b']]


By the way, the Monoid, above, and others like it of different orders, can be automatically generated using the function, ``generate_commutative_monoid``.  It is based on integer multiplication modulo the desired order.

In [25]:
>>> m4.identity  # Returns the identity element

'b'

In [26]:
>>> m4.op('c', 'b')  # since 'b' is the identity element

'c'

## Ring

**Ring** -- $\langle S, +, \times \rangle$, where $\langle S, + \rangle$ is a commutative Group, $\langle S, \times \rangle$ is a Semigroup, and $\times$ distributes over $+$

### Ring Based on Powerset of a Set

In this ring, *"addition"* is symmetric difference, $\bigtriangleup$, and *"multiplication"* is intersection, $\cap$.

In [27]:
>>> rng = make_finite_algebra('Powerset Ring 2',
                              'Ring on powerset of {0, 1}',
                              ['{}', '{0}', '{1}', '{0, 1}'],
                              [[0, 1, 2, 3],
                               [1, 0, 3, 2],
                               [2, 3, 0, 1],
                               [3, 2, 1, 0]],
                              [[0, 0, 0, 0],
                               [0, 1, 0, 1],
                               [0, 0, 2, 2],
                               [0, 1, 2, 3]]
                             )
>>> rng

Ring(
'Powerset Ring 2',
'Ring on powerset of {0, 1}',
['{}', '{0}', '{1}', '{0, 1}'],
[[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 0, 1], [3, 2, 1, 0]],
[[0, 0, 0, 0], [0, 1, 0, 1], [0, 0, 2, 2], [0, 1, 2, 3]]
)

In [28]:
>>> rng.about(use_table_names=True)


Ring: Powerset Ring 2
Instance ID: 140320620175440
Description: Ring on powerset of {0, 1}
Order: 4
Identity: {}
Associative? Yes
Commutative? Yes
Elements:
   Index   Name   Inverse  Order
      0      {}      {}       1
      1     {0}     {0}       2
      2     {1}     {1}       2
      3  {0, 1}  {0, 1}       2
Cayley Table (showing names):
[['{}', '{0}', '{1}', '{0, 1}'],
 ['{0}', '{}', '{0, 1}', '{1}'],
 ['{1}', '{0, 1}', '{}', '{0}'],
 ['{0, 1}', '{1}', '{0}', '{}']]
Mult. Identity: {0, 1}
Mult. Commutative? Yes
Multiplicative Cayley Table (showing names):
[['{}', '{}', '{}', '{}'],
 ['{}', '{0}', '{}', '{0}'],
 ['{}', '{}', '{1}', '{1}'],
 ['{}', '{0}', '{1}', '{0, 1}']]


### Ring Addition and Multiplication

Ring addition, ``add``, is the same as the operation, ``op``, inherited from its superclass, Group.

$\{1\} \bigtriangleup \{0,1\} = \{0\}$

In [29]:
>>> rng.add("{1}", "{0, 1}")

'{0}'

$\{1\} \cap \{0,1\} = \{0\}$

In [30]:
>>> rng.mult("{1}", "{0, 1}")

'{1}'

### Zero Divisors of a Ring

Suppose $\alpha \ne 0$ is an element of the Ring, $\langle S, +, \times \rangle$.

Then, $\alpha$ is a **left zero divisor**, if $\exists \beta \in S, \beta \ne 0$ such that $\alpha \times \beta = 0$.

Similarly, $\alpha$ is a **right zero divisor**, if $\exists \gamma \in S, \gamma \ne 0$ such that $\gamma \times \alpha = 0$.

The Ring just created has two zero divisors:

In [31]:
>>> rng.zero_divisors()

['{0}', '{1}']

To check this, recall, what the addititve identity is:

In [32]:
zero = rng.add_identity
zero

'{}'

Multiplying an element by "zero" produces "zero":

In [33]:
[rng.mult(x, zero) for x in rng.elements]

['{}', '{}', '{}', '{}']

In [34]:
[rng.mult(x, '{0}') for x in rng.elements]

['{}', '{0}', '{}', '{0}']

In [35]:
rng.mult('{0}', '{1}')

'{}'

### Autogeneration of a Powerset Ring

In [36]:
>>> from finite_algebras import generate_powerset_ring

>>> psr3 = generate_powerset_ring(3)  # Ring order will be 3!

>>> psr3

Ring(
'PSRing3',
'Autogenerated Ring on powerset of {0, 1, 2} w/ symm. diff. (add) & intersection (mult)',
['{}', '{0}', '{1}', '{2}', '{0, 1}', '{0, 2}', '{1, 2}', '{0, 1, 2}'],
[[0, 1, 2, 3, 4, 5, 6, 7], [1, 0, 4, 5, 2, 3, 7, 6], [2, 4, 0, 6, 1, 7, 3, 5], [3, 5, 6, 0, 7, 1, 2, 4], [4, 2, 1, 7, 0, 6, 5, 3], [5, 3, 7, 1, 6, 0, 4, 2], [6, 7, 3, 2, 5, 4, 0, 1], [7, 6, 5, 4, 3, 2, 1, 0]],
[[0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 1, 1, 0, 1], [0, 0, 2, 0, 2, 0, 2, 2], [0, 0, 0, 3, 0, 3, 3, 3], [0, 1, 2, 0, 4, 1, 2, 4], [0, 1, 0, 3, 1, 5, 3, 5], [0, 0, 2, 3, 2, 3, 6, 6], [0, 1, 2, 3, 4, 5, 6, 7]]
)

In [37]:
>>> psr3.about(use_table_names=True)


Ring: PSRing3
Instance ID: 140321287566992
Description: Autogenerated Ring on powerset of {0, 1, 2} w/ symm. diff. (add) & intersection (mult)
Order: 8
Identity: {}
Associative? Yes
Commutative? Yes
Elements:
   Index   Name   Inverse  Order
      0      {}      {}       1
      1     {0}     {0}       2
      2     {1}     {1}       2
      3     {2}     {2}       2
      4  {0, 1}  {0, 1}       2
      5  {0, 2}  {0, 2}       2
      6  {1, 2}  {1, 2}       2
      7 {0, 1, 2} {0, 1, 2}       2
Cayley Table (showing names):
[['{}', '{0}', '{1}', '{2}', '{0, 1}', '{0, 2}', '{1, 2}', '{0, 1, 2}'],
 ['{0}', '{}', '{0, 1}', '{0, 2}', '{1}', '{2}', '{0, 1, 2}', '{1, 2}'],
 ['{1}', '{0, 1}', '{}', '{1, 2}', '{0}', '{0, 1, 2}', '{2}', '{0, 2}'],
 ['{2}', '{0, 2}', '{1, 2}', '{}', '{0, 1, 2}', '{0}', '{1}', '{0, 1}'],
 ['{0, 1}', '{1}', '{0}', '{0, 1, 2}', '{}', '{1, 2}', '{0, 2}', '{2}'],
 ['{0, 2}', '{2}', '{0, 1, 2}', '{0}', '{1, 2}', '{}', '{0, 1}', '{1}'],
 ['{1, 2}', '{0, 1, 2}', '{2}

### Ring Based on 2x2 Matrices

See Example 6 in this reference: http://www-groups.mcs.st-andrews.ac.uk/~john/MT4517/Lectures/L3.html

Example 6 is a Ring based on the following matrices, where arithmetic is done modulo 2:

$0 = \begin{bmatrix} 0 & 0 \\ 0 & 0 \end{bmatrix}, a = \begin{bmatrix} 0 & 1 \\ 0 & 0 \end{bmatrix}, b = \begin{bmatrix} 0 & 1 \\ 0 & 1 \end{bmatrix}, c = \begin{bmatrix} 0 & 0 \\ 0 & 1 \end{bmatrix}$

In [38]:
>>> addtbl = [['0', 'a', 'b', 'c'],
              ['a', '0', 'c', 'b'],
              ['b', 'c', '0', 'a'],
              ['c', 'b', 'a', '0']]

>>> multbl = [['0', '0', '0', '0'],
              ['0', '0', 'a', 'a'],
              ['0', '0', 'b', 'b'],
              ['0', '0', 'c', 'c']]

>>> ex6 = make_finite_algebra(
    'Ex6',
    'Example 6: http://www-groups.mcs.st-andrews.ac.uk/~john/MT4517/Lectures/L3.html',
    ['0', 'a', 'b', 'c'],
    addtbl,
    multbl)

>>> ex6

Ring(
'Ex6',
'Example 6: http://www-groups.mcs.st-andrews.ac.uk/~john/MT4517/Lectures/L3.html',
['0', 'a', 'b', 'c'],
[[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 0, 1], [3, 2, 1, 0]],
[[0, 0, 0, 0], [0, 0, 1, 1], [0, 0, 2, 2], [0, 0, 3, 3]]
)

In [39]:
>>> ex6.about(use_table_names=True)


Ring: Ex6
Instance ID: 140320620178064
Description: Example 6: http://www-groups.mcs.st-andrews.ac.uk/~john/MT4517/Lectures/L3.html
Order: 4
Identity: 0
Associative? Yes
Commutative? Yes
Elements:
   Index   Name   Inverse  Order
      0       0       0       1
      1       a       a       2
      2       b       b       2
      3       c       c       2
Cayley Table (showing names):
[['0', 'a', 'b', 'c'],
 ['a', '0', 'c', 'b'],
 ['b', 'c', '0', 'a'],
 ['c', 'b', 'a', '0']]
Mult. Identity: None
Mult. Commutative? No
Multiplicative Cayley Table (showing names):
[['0', '0', '0', '0'],
 ['0', '0', 'a', 'a'],
 ['0', '0', 'b', 'b'],
 ['0', '0', 'c', 'c']]


### Extracting a Ring's Additive & Multiplicative "Subalgebras"

At the beginning of this User Guide, in the *Algebra Definitions* section, a Ring is described as being a combination of a commutative Group, under addition, and a Semigroup, under multiplication (with distributivity of multiplication over addition). This section shows how those algebraic components of a Ring can be extracted.

**NOTE**: The implementation of the two extraction methods, illustrated below, operates by calling ``make_finite_algebra`` using the relevant portions of the Ring.  That way, the appropriate algebras are returned: a commutative Group for the additive portion, and, at a minimum, a Semigroup for the multiplicative portion.

In [40]:
>>> ex6

Ring(
'Ex6',
'Example 6: http://www-groups.mcs.st-andrews.ac.uk/~john/MT4517/Lectures/L3.html',
['0', 'a', 'b', 'c'],
[[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 0, 1], [3, 2, 1, 0]],
[[0, 0, 0, 0], [0, 0, 1, 1], [0, 0, 2, 2], [0, 0, 3, 3]]
)

The **additive portion** of this example ring is a commutative Group, as expected:

In [41]:
>>> ex6_add = ex6.extract_additive_algebra()
>>> ex6_add.about()


Group: Ex6.Add
Instance ID: 140321287619216
Description: Additive-only portion of Ex6
Order: 4
Identity: 0
Associative? Yes
Commutative? Yes
Elements:
   Index   Name   Inverse  Order
      0       0       0       1
      1       a       a       2
      2       b       b       2
      3       c       c       2
Cayley Table (showing indices):
[[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 0, 1], [3, 2, 1, 0]]


And, the **multiplicative portion** is a Semigroup:

In [42]:
>>> ex6_mult = ex6.extract_multiplicative_algebra()
>>> ex6_mult

Semigroup(
'Ex6.Mult',
'Multiplicative-only portion of Ex6',
['0', 'a', 'b', 'c'],
[[0, 0, 0, 0], [0, 0, 1, 1], [0, 0, 2, 2], [0, 0, 3, 3]]
)

### Autogenerating a Commutative Ring

The function, ``generate_algebra_mod_n``, is based on [example 2 here](http://www-groups.mcs.st-andrews.ac.uk/~john/MT4517/Lectures/L3.html) and in [Wikipedia here](https://en.wikipedia.org/wiki/Finite_field#Field_with_four_elements).  The $+$ and $\times$ operations are the usual integer addition and multiplication modulo the order (n), resp.

As long as the order (n) is not prime the function ``generate_algebra_mod_n`` will produce a Ring, but for a prime order, it will produce a Field.

In [43]:
>>> from finite_algebras import generate_algebra_mod_n

In [44]:
>>> r6 = generate_algebra_mod_n(6)
>>> r6

Ring(
'R6',
'Autogenerated Ring of integers mod 6',
['a0', 'a1', 'a2', 'a3', 'a4', 'a5'],
[[0, 1, 2, 3, 4, 5], [1, 2, 3, 4, 5, 0], [2, 3, 4, 5, 0, 1], [3, 4, 5, 0, 1, 2], [4, 5, 0, 1, 2, 3], [5, 0, 1, 2, 3, 4]],
[[0, 0, 0, 0, 0, 0], [0, 1, 2, 3, 4, 5], [0, 2, 4, 0, 2, 4], [0, 3, 0, 3, 0, 3], [0, 4, 2, 0, 4, 2], [0, 5, 4, 3, 2, 1]]
)

In [45]:
>>> r6.about(use_table_names=True)


Ring: R6
Instance ID: 140321287632720
Description: Autogenerated Ring of integers mod 6
Order: 6
Identity: a0
Associative? Yes
Commutative? Yes
Elements:
   Index   Name   Inverse  Order
      0      a0      a0       1
      1      a1      a5       6
      2      a2      a4       3
      3      a3      a3       2
      4      a4      a2       3
      5      a5      a1       6
Cayley Table (showing names):
[['a0', 'a1', 'a2', 'a3', 'a4', 'a5'],
 ['a1', 'a2', 'a3', 'a4', 'a5', 'a0'],
 ['a2', 'a3', 'a4', 'a5', 'a0', 'a1'],
 ['a3', 'a4', 'a5', 'a0', 'a1', 'a2'],
 ['a4', 'a5', 'a0', 'a1', 'a2', 'a3'],
 ['a5', 'a0', 'a1', 'a2', 'a3', 'a4']]
Mult. Identity: a1
Mult. Commutative? Yes
Multiplicative Cayley Table (showing names):
[['a0', 'a0', 'a0', 'a0', 'a0', 'a0'],
 ['a0', 'a1', 'a2', 'a3', 'a4', 'a5'],
 ['a0', 'a2', 'a4', 'a0', 'a2', 'a4'],
 ['a0', 'a3', 'a0', 'a3', 'a0', 'a3'],
 ['a0', 'a4', 'a2', 'a0', 'a4', 'a2'],
 ['a0', 'a5', 'a4', 'a3', 'a2', 'a1']]


**Extracting it's component algebras**

In the following, we extract the component algebras of this Ring as a commutative Group and a Monoid.

The Monoid occurs since this Ring's multiplicative portion includes a multiplicative identity element ('a1'), but does not include inverses of all elements.

In [46]:
>>> r6add = r6.extract_additive_algebra()
>>> r6add.about()


Group: R6.Add
Instance ID: 140321287610896
Description: Additive-only portion of R6
Order: 6
Identity: a0
Associative? Yes
Commutative? Yes
Elements:
   Index   Name   Inverse  Order
      0      a0      a0       1
      1      a1      a5       6
      2      a2      a4       3
      3      a3      a3       2
      4      a4      a2       3
      5      a5      a1       6
Cayley Table (showing indices):
[[0, 1, 2, 3, 4, 5],
 [1, 2, 3, 4, 5, 0],
 [2, 3, 4, 5, 0, 1],
 [3, 4, 5, 0, 1, 2],
 [4, 5, 0, 1, 2, 3],
 [5, 0, 1, 2, 3, 4]]


In [47]:
>>> r6mult = r6.extract_multiplicative_algebra()
>>> r6mult.about()


Monoid: R6.Mult
Instance ID: 140321287633552
Description: Multiplicative-only portion of R6
Order: 6
Elements: ['a0', 'a1', 'a2', 'a3', 'a4', 'a5']
Identity: a1
Associative? Yes
Commutative? Yes
Has Inverses? No
Cayley Table (showing indices):
[[0, 0, 0, 0, 0, 0],
 [0, 1, 2, 3, 4, 5],
 [0, 2, 4, 0, 2, 4],
 [0, 3, 0, 3, 0, 3],
 [0, 4, 2, 0, 4, 2],
 [0, 5, 4, 3, 2, 1]]


## Field

**Field** -- a Ring $\langle S, +, \times \rangle$, where $\langle S\setminus{\{0\}}, \times \rangle$ is a commutative Group.

$S\setminus{\{0\}}$ is the set $S$ with the additive identity element removed.

### Field with four elements

**Reference**: See Wikipedia: ["Field with four elements"](https://en.wikipedia.org/wiki/Finite_field#Field_with_four_elements)

In [48]:
>>> elems = ['0', '1', 'a', '1+a']

>>> add_table = [[ '0' ,  '1' ,  'a' , '1+a'],
                 [ '1' ,  '0' , '1+a',  'a' ],
                 [ 'a' , '1+a',  '0' ,  '1' ],
                 ['1+a',  'a' ,  '1' ,  '0' ]]

>>> mult_table = [['0',  '0' ,  '0' ,  '0' ],
                  ['0',  '1' ,  'a' , '1+a'],
                  ['0',  'a' , '1+a',  '1' ],
                  ['0', '1+a',  '1' ,  'a' ]]

>>> f4 = make_finite_algebra('F4',
                             'Field with 4 elements',
                             elems,
                             add_table,
                             mult_table
                            )
>>> f4.about()


Field: F4
Instance ID: 140321287668944
Description: Field with 4 elements
Order: 4
Identity: 0
Associative? Yes
Commutative? Yes
Elements:
   Index   Name   Inverse  Order
      0       0       0       1
      1       1       1       2
      2       a       a       2
      3     1+a     1+a       2
Cayley Table (showing indices):
[[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 0, 1], [3, 2, 1, 0]]
Mult. Identity: 1
Mult. Commutative? Yes
Multiplicative Cayley Table (showing indices):
[[0, 0, 0, 0], [0, 1, 2, 3], [0, 2, 3, 1], [0, 3, 1, 2]]


### Addition & Multiplication in Fields

A Field's addition and multiplication operations are inherited from its superclass, Ring.

In [49]:
>>> f4.add('a', '1')

'1+a'

In [50]:
>>> f4.mult('a', 'a')

'1+a'

### Division in Fields

The method, ``div``, is a convenience method in Fields for computing "$\alpha \div \beta, \beta \ne 0$", that is, $\alpha \times \beta^{-1}$ where $\alpha, \beta \in \langle F, +, \times \rangle$.

In [51]:
>>> a = 'a'
>>> b = '1+a'
>>> print(f"For example, \"{a} / {b}\" = {a} * {f4.mult_inv(b)} = {f4.mult(a, f4.mult_inv(b))}")

For example, "a / 1+a" = a * a = 1+a


In [52]:
>>> f4.div(a, b)

'1+a'

Recall the definition of a Field, given at the beginning of this User Guide:

**Field** -- a Ring $\langle S, +, \times \rangle$, where $\langle S\setminus{\{0\}}, \times \rangle$ is a commutative Group.

During Field construction, the commutative Group, mentioned in the definition, is also constructed and stored inside the Field instance.  It is used to obtain multiplicative inverses and to define a *division* method, ``div``.

The ``div`` method, for example, can be used to construct the "Division" table shown in the Wikipedia entry, ["Field with four elements"](https://en.wikipedia.org/wiki/Finite_field#Field_with_four_elements):

In [53]:
>>> div_table = [[f4.div(x, y) for y in f4.elements] for x in f4.elements]
>>> div_table

[[None, '0', '0', '0'],
 [None, '1', '1+a', 'a'],
 [None, 'a', '1', '1+a'],
 [None, '1+a', 'a', '1']]

### Autogenerated Prime Field

The example here uses the function, ``generate_algebra_mod_n``, described above.  As noted above, if the order, n, is prime, then it will produce a Field.

In [54]:
>>> from finite_algebras import generate_algebra_mod_n

>>> f7 = generate_algebra_mod_n(7)
>>> f7.about()


Field: F7
Instance ID: 140321287711312
Description: Autogenerated Field of integers mod 7
Order: 7
Identity: a0
Associative? Yes
Commutative? Yes
Elements:
   Index   Name   Inverse  Order
      0      a0      a0       1
      1      a1      a6       7
      2      a2      a5       7
      3      a3      a4       7
      4      a4      a3       7
      5      a5      a2       7
      6      a6      a1       7
Cayley Table (showing indices):
[[0, 1, 2, 3, 4, 5, 6],
 [1, 2, 3, 4, 5, 6, 0],
 [2, 3, 4, 5, 6, 0, 1],
 [3, 4, 5, 6, 0, 1, 2],
 [4, 5, 6, 0, 1, 2, 3],
 [5, 6, 0, 1, 2, 3, 4],
 [6, 0, 1, 2, 3, 4, 5]]
Mult. Identity: a1
Mult. Commutative? Yes
Multiplicative Cayley Table (showing indices):
[[0, 0, 0, 0, 0, 0, 0],
 [0, 1, 2, 3, 4, 5, 6],
 [0, 2, 4, 6, 1, 3, 5],
 [0, 3, 6, 2, 5, 1, 4],
 [0, 4, 1, 5, 2, 6, 3],
 [0, 5, 3, 1, 6, 4, 2],
 [0, 6, 5, 4, 3, 2, 1]]


## Serialization

Algebras can be converted to and from JSON strings/files and Python dictionaries.

### Instantiate Algebra from JSON File

First setup some path variables:

* one that points to the abstract_algebra directory
* and the other points to a subdirectory containing algebra definitions in JSON format

Also, the code here assumes that there is an environment
variable, ``PYPROJ``, that points to the parent directory of the abstract_algebra directory.

In [55]:
>>> import os
>>> aa_path = os.path.join(os.getenv("PYPROJ"), "abstract_algebra")
>>> alg_dir = os.path.join(aa_path, "Algebras")

Here's the **JSON file**:

In [56]:
>>> v4_json = os.path.join(alg_dir, "v4_klein_4_group.json")

>>> !cat {v4_json}

{"name": "V4",
 "description": "Klein-4 group",
 "elements": ["e", "h", "v", "r"],
 "table": [[0, 1, 2, 3],
           [1, 0, 3, 2],
           [2, 3, 0, 1],
           [3, 2, 1, 0]]
}


And, here's the **algebra** that is loaded from the JSON file:

In [57]:
>>> v4 = make_finite_algebra(v4_json)

>>> v4

Group(
'V4',
'Klein-4 group',
['e', 'h', 'v', 'r'],
[[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 0, 1], [3, 2, 1, 0]]
)

### Convert Algebra to Python Dictionary

The examples, below, show a Magma, Group, & Field, being converted into dictionaries.

In [58]:
>>> rps.to_dict()

{'name': 'RPS',
 'description': 'Rock, Paper, Scissors Magma',
 'elements': ['r', 'p', 's'],
 'table': [[0, 1, 0], [1, 1, 2], [0, 2, 2]]}

The **type** of algebra (e.g., Magma) can be included in the dictionary for readability, however, the *type* field is ignored when ``make_finite_algebra`` reads a dictionary or JSON file.

In [59]:
>>> rps_dict = rps.to_dict(include_classname=True)

>>> rps_dict

{'name': 'RPS',
 'description': 'Rock, Paper, Scissors Magma',
 'elements': ['r', 'p', 's'],
 'table': [[0, 1, 0], [1, 1, 2], [0, 2, 2]],
 'type': 'Magma'}

In [60]:
>>> v4_dict = v4.to_dict()

>>> v4_dict

{'name': 'V4',
 'description': 'Klein-4 group',
 'elements': ['e', 'h', 'v', 'r'],
 'table': [[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 0, 1], [3, 2, 1, 0]]}

In [61]:
>>> f4_dict = f4.to_dict()

>>> f4_dict

{'name': 'F4',
 'description': 'Field with 4 elements',
 'elements': ['0', '1', 'a', '1+a'],
 'table': [[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 0, 1], [3, 2, 1, 0]],
 'table2': [[0, 0, 0, 0], [0, 1, 2, 3], [0, 2, 3, 1], [0, 3, 1, 2]]}

### Instantiate Algebra from Python Dictionary

In [62]:
>>> rps_from_dict = make_finite_algebra(rps_dict)

>>> rps_from_dict

Magma(
'RPS',
'Rock, Paper, Scissors Magma',
['r', 'p', 's'],
[[0, 1, 0], [1, 1, 2], [0, 2, 2]]
)

In [63]:
>>> v4_from_dict = make_finite_algebra(v4_dict)

>>> v4_from_dict

Group(
'V4',
'Klein-4 group',
['e', 'h', 'v', 'r'],
[[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 0, 1], [3, 2, 1, 0]]
)

In [64]:
>>> f4_from_dict = make_finite_algebra(f4_dict)

>>> f4_from_dict

Field(
'F4',
'Field with 4 elements',
['0', '1', 'a', '1+a'],
[[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 0, 1], [3, 2, 1, 0]],
[[0, 0, 0, 0], [0, 1, 2, 3], [0, 2, 3, 1], [0, 3, 1, 2]]
)

### Convert Algebra to JSON String

In [65]:
>>> v4_json_string = v4.dumps()

>>> v4_json_string

'{"name": "V4", "description": "Klein-4 group", "elements": ["e", "h", "v", "r"], "table": [[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 0, 1], [3, 2, 1, 0]]}'

**WARNING**: Although an algebra can be constructed by loading its definition from a JSON file, it cannot be constructed directly from a JSON string, because ``make_finite_algebra`` interprets a single string input as a JSON file name.  To load an algebra from a JSON string, first convert the string to a Python dictionary, then input that to ``make_finite_algebra``, as shown below:

In [66]:
>>> import json

>>> make_finite_algebra(json.loads(v4_json_string))

Group(
'V4',
'Klein-4 group',
['e', 'h', 'v', 'r'],
[[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 0, 1], [3, 2, 1, 0]]
)

## Autogeneration of Finite Algebras

There are several functions for autogenerating finite algebras of specified size:

**Groups**

* ``generate_cyclic_group(n)``: $Z_n$, where $a \circ b \equiv a+b$ mod $n$, where $a,b \in \{0,1,...,n-1\}$
* ``generate_symmetric_group(n)``: $S_n$, where $\circ$ is composition of permutations of $(0, 1, ..., n-1)$
* ``generate_powerset_group(n)``: $A \circ B \equiv A \bigtriangleup B$, where $A,B \in P(\{0, 1, ..., n-1\})$; order is $2^n$

**Monoid**

* ``generate_commutative_monoid(n)``: $a \circ b \equiv ab$ mod $n$, where $a,b \in \{0,1,...,n-1\}$

**Rings & Fields**

* ``generate_powerset_ring``: $A+B \equiv A \bigtriangleup B$ and $A \times B \equiv A \cap B$, where $A,B \in P(\{0, 1, ..., n-1\})$
* ``generate_algebra_mod_n``: Combination of <i>generate_cyclic_group</i> ($+$) and <i>generate_commutative_monoid</i> ($\times$)
  * If n is prime, then this will be a Field, otherwise it will be a Ring

### Autogenerated Cyclic Group

A cyclic group of any desired order can be generated as follows:

In [67]:
>>> from finite_algebras import generate_cyclic_group

>>> z2 = generate_cyclic_group(2)

>>> z2.about()


Group: Z2
Instance ID: 140321287619728
Description: Autogenerated cyclic Group of order 2
Order: 2
Identity: e
Associative? Yes
Commutative? Yes
Elements:
   Index   Name   Inverse  Order
      0       e       e       1
      1       a       a       2
Cayley Table (showing indices):
[[0, 1], [1, 0]]


### Autogenerated Symmetric Group

The symmetric group, based on the permutations of <b>n</b> elements, (1, 2, 3, ..., n), can be generated as follows:

**WARNING**: Since the order of an autogenerated symmetric group is **n!**, even a small value of **n** can result in a very large group.

In [68]:
>>> from finite_algebras import generate_symmetric_group

>>> s3 = generate_symmetric_group(3)

>>> s3.about()


Group: S3
Instance ID: 140321287678416
Description: Autogenerated symmetric Group on 3 elements
Order: 6
Identity: (1, 2, 3)
Associative? Yes
Commutative? No
Elements:
   Index   Name   Inverse  Order
      0 (1, 2, 3) (1, 2, 3)       1
      1 (1, 3, 2) (1, 3, 2)       2
      2 (2, 1, 3) (2, 1, 3)       2
      3 (2, 3, 1) (3, 1, 2)       3
      4 (3, 1, 2) (2, 3, 1)       3
      5 (3, 2, 1) (3, 2, 1)       2
Cayley Table (showing indices):
[[0, 1, 2, 3, 4, 5],
 [1, 0, 4, 5, 2, 3],
 [2, 3, 0, 1, 5, 4],
 [3, 2, 5, 4, 0, 1],
 [4, 5, 1, 0, 3, 2],
 [5, 4, 3, 2, 1, 0]]


### Autogenerated Powerset Group

The function, ``generate_powerset_group``, will generate a group on the powerset of {0, 1, 2, ..., n-1} with **symmetric difference** as the group's binary operation.  This group is useful because it can be used to form a ring with set intersection as the second operator.

This means that the order of the autogenerated powerset group will be $2^n$, so the same WARNING as above applies with regard to large values of n.

In [69]:
>>> from finite_algebras import generate_powerset_group

>>> ps3 = generate_powerset_group(3)

>>> ps3.about()


Group: PS3
Instance ID: 140321287619408
Description: Autogenerated Group on the powerset of 3 elements, with symmetric difference operator
Order: 8
Identity: {}
Associative? Yes
Commutative? Yes
Elements:
   Index   Name   Inverse  Order
      0      {}      {}       1
      1     {0}     {0}       2
      2     {1}     {1}       2
      3     {2}     {2}       2
      4  {0, 1}  {0, 1}       2
      5  {0, 2}  {0, 2}       2
      6  {1, 2}  {1, 2}       2
      7 {0, 1, 2} {0, 1, 2}       2
Cayley Table (showing indices):
[[0, 1, 2, 3, 4, 5, 6, 7],
 [1, 0, 4, 5, 2, 3, 7, 6],
 [2, 4, 0, 6, 1, 7, 3, 5],
 [3, 5, 6, 0, 7, 1, 2, 4],
 [4, 2, 1, 7, 0, 6, 5, 3],
 [5, 3, 7, 1, 6, 0, 4, 2],
 [6, 7, 3, 2, 5, 4, 0, 1],
 [7, 6, 5, 4, 3, 2, 1, 0]]


### Autogenerated Monoid

The function, ``generate_commutative_monoid``, is based on integer multiplication modulo the desired order.

In [70]:
>>> from finite_algebras import generate_commutative_monoid

>>> m7 = generate_commutative_monoid(7)

>>> m7.about()


Monoid: M7
Instance ID: 140320620176528
Description: Autogenerated commutative Monoid of order 7
Order: 7
Elements: ['a0', 'a1', 'a2', 'a3', 'a4', 'a5', 'a6']
Identity: a1
Associative? Yes
Commutative? Yes
Has Inverses? No
Cayley Table (showing indices):
[[0, 0, 0, 0, 0, 0, 0],
 [0, 1, 2, 3, 4, 5, 6],
 [0, 2, 4, 6, 1, 3, 5],
 [0, 3, 6, 2, 5, 1, 4],
 [0, 4, 1, 5, 2, 6, 3],
 [0, 5, 3, 1, 6, 4, 2],
 [0, 6, 5, 4, 3, 2, 1]]


## Direct Products

The **direct product** of two or more algebras can be generated using Python's multiplication operator, ``*``:

### Direct Product of Multiple Groups

In [71]:
>>> z2_cubed = z2 * z2 * z2

>>> z2_cubed.about()


Group: Z2_x_Z2_x_Z2
Instance ID: 140320620268048
Description: Direct product of Z2_x_Z2 & Z2
Order: 8
Identity: e:e:e
Associative? Yes
Commutative? Yes
Elements:
   Index   Name   Inverse  Order
      0   e:e:e   e:e:e       1
      1   e:e:a   e:e:a       2
      2   e:a:e   e:a:e       2
      3   e:a:a   e:a:a       2
      4   a:e:e   a:e:e       2
      5   a:e:a   a:e:a       2
      6   a:a:e   a:a:e       2
      7   a:a:a   a:a:a       2
Cayley Table (showing indices):
[[0, 1, 2, 3, 4, 5, 6, 7],
 [1, 0, 3, 2, 5, 4, 7, 6],
 [2, 3, 0, 1, 6, 7, 4, 5],
 [3, 2, 1, 0, 7, 6, 5, 4],
 [4, 5, 6, 7, 0, 1, 2, 3],
 [5, 4, 7, 6, 1, 0, 3, 2],
 [6, 7, 4, 5, 2, 3, 0, 1],
 [7, 6, 5, 4, 3, 2, 1, 0]]


### Direct Product of Monoids

In [72]:
>>> mon3 = generate_commutative_monoid(3)

>>> mon3

Monoid(
'M3',
'Autogenerated commutative Monoid of order 3',
['a0', 'a1', 'a2'],
[[0, 0, 0], [0, 1, 2], [0, 2, 1]]
)

In [73]:
>>> m3_sqr = mon3 * mon3
>>> m3_sqr.about()


Monoid: M3_x_M3
Instance ID: 140321287621712
Description: Direct product of M3 & M3
Order: 9
Elements: ['a0:a0', 'a0:a1', 'a0:a2', 'a1:a0', 'a1:a1', 'a1:a2', 'a2:a0', 'a2:a1', 'a2:a2']
Identity: a1:a1
Associative? Yes
Commutative? Yes
Has Inverses? No
Cayley Table (showing indices):
[[0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 2, 0, 1, 2, 0, 1, 2],
 [0, 2, 1, 0, 2, 1, 0, 2, 1],
 [0, 0, 0, 3, 3, 3, 6, 6, 6],
 [0, 1, 2, 3, 4, 5, 6, 7, 8],
 [0, 2, 1, 3, 5, 4, 6, 8, 7],
 [0, 0, 0, 6, 6, 6, 3, 3, 3],
 [0, 1, 2, 6, 7, 8, 3, 4, 5],
 [0, 2, 1, 6, 8, 7, 3, 5, 4]]


## Isomorphisms

If two groups are isomorphic, then the mapping between their elements is returned as a Python dictionary.

Here'a a well-known example, using two small groups created above:

### Group Isomorphism

In [74]:
>>> z2_sqr = z2 * z2

>>> v4.isomorphic(z2_sqr)

{'e': 'e:e', 'h': 'e:a', 'v': 'a:e', 'r': 'a:a'}

If two groups are not isomorphic, then ``False`` is returned.

In [75]:
>>> z4 = generate_cyclic_group(4)

>>> z4.isomorphic(z2_sqr)

False

### Magma Isomorphism

**Water, Fire, Stick Magma**

A made-up Magma, similar to Rock, Paper, Scissors:

* Water quenches Fire
* Fire burns Stick
* Stick floats on Water

In [76]:
>>> wfs = make_finite_algebra('WFS',
                              'Water, Fire, Stick Magma',
                              ['water', 'fire', 'stick'],
                              [[0, 0, 2],
                               [0, 1, 1],
                               [2, 1, 2]])
>>> wfs

Magma(
'WFS',
'Water, Fire, Stick Magma',
['water', 'fire', 'stick'],
[[0, 0, 2], [0, 1, 1], [2, 1, 2]]
)

Here's the isomorphism between rps and wfs:

In [77]:
>>> rps.isomorphic(wfs)

{'r': 'water', 'p': 'stick', 's': 'fire'}

## Subalgebras (Subgroups)

A Group can contain subgroups, submonoids, subsemigroups, or submagmas.  In general, all of these are referred to here as *subalgebras*.

The method, ``proper_subalgebras``, extracts all possible subalgebras that exist within an algebra, regardless of whether they are isomorphic to each other or not, or even of the same algebraic class as the parent algebra.

### Proper Subgroups

In [78]:
>>> z8 = generate_cyclic_group(8)
>>> z8.about()


Group: Z8
Instance ID: 140321287582352
Description: Autogenerated cyclic Group of order 8
Order: 8
Identity: e
Associative? Yes
Commutative? Yes
Elements:
   Index   Name   Inverse  Order
      0       e       e       1
      1       a     a^7       8
      2     a^2     a^6       4
      3     a^3     a^5       8
      4     a^4     a^4       2
      5     a^5     a^3       8
      6     a^6     a^2       4
      7     a^7       a       8
Cayley Table (showing indices):
[[0, 1, 2, 3, 4, 5, 6, 7],
 [1, 2, 3, 4, 5, 6, 7, 0],
 [2, 3, 4, 5, 6, 7, 0, 1],
 [3, 4, 5, 6, 7, 0, 1, 2],
 [4, 5, 6, 7, 0, 1, 2, 3],
 [5, 6, 7, 0, 1, 2, 3, 4],
 [6, 7, 0, 1, 2, 3, 4, 5],
 [7, 0, 1, 2, 3, 4, 5, 6]]


In [79]:
>>> z8_proper_subs = z8.proper_subalgebras()

>>> _ = [z8_proper_sub.about() for z8_proper_sub in z8_proper_subs]


Group: Z8_subalgebra_0
Instance ID: 140321287611664
Description: Subalgebra of: Autogenerated cyclic Group of order 8
Order: 2
Identity: e
Associative? Yes
Commutative? Yes
Elements:
   Index   Name   Inverse  Order
      0       e       e       1
      1     a^4     a^4       2
Cayley Table (showing indices):
[[0, 1], [1, 0]]

Group: Z8_subalgebra_1
Instance ID: 140321287577872
Description: Subalgebra of: Autogenerated cyclic Group of order 8
Order: 4
Identity: e
Associative? Yes
Commutative? Yes
Elements:
   Index   Name   Inverse  Order
      0       e       e       1
      1     a^2     a^6       4
      2     a^4     a^4       2
      3     a^6     a^2       4
Cayley Table (showing indices):
[[0, 1, 2, 3], [1, 2, 3, 0], [2, 3, 0, 1], [3, 0, 1, 2]]


### Normal Subgroups

Both of the subgroups of Z8, derived above, are **normal**:

In [80]:
>>> [z8.is_normal(g) for g in z8_proper_subs]

[True, True]

### Proper Subalgebras up to Isomorphism

The function, ``partition_into_isomorphic_lists``, does just that; it partitions a list of algebras (subgroups in this case) into a list of lists, where each sublist contains subalgebras that are all isomophic to each other.

The function, ``about_isomorphic_partitions``, prints out a summary of information about the partitions output by ``partition_into_isomorphic_list``.

In [81]:
from finite_algebras import partition_into_isomorphic_lists, about_isomorphic_partitions

The example, below, uses the autogenerated powerset group, **ps3**, that was created earlier.

In [82]:
>>> ps3_proper_subs = ps3.proper_subalgebras()

>>> partitions = partition_into_isomorphic_lists(ps3_proper_subs)

>>> about_isomorphic_partitions(ps3, partitions)


Subalgebras of <Group:PS3, ID:140321287619408>
  There are 2 unique subalgebras, up to isomorphisms, out of 14 total subalgebras
  as shown by the partitions below:

7 Commutative Normal Groups of Order 4 with identity '{}':
      PS3_subalgebra_0: ['{}', '{0}', '{1, 2}', '{0, 1, 2}']
      PS3_subalgebra_1: ['{}', '{1}', '{0, 2}', '{0, 1, 2}']
      PS3_subalgebra_4: ['{}', '{0}', '{2}', '{0, 2}']
      PS3_subalgebra_6: ['{}', '{0, 1}', '{0, 2}', '{1, 2}']
      PS3_subalgebra_10: ['{}', '{0}', '{1}', '{0, 1}']
      PS3_subalgebra_12: ['{}', '{2}', '{0, 1}', '{0, 1, 2}']
      PS3_subalgebra_13: ['{}', '{1}', '{2}', '{1, 2}']

7 Commutative Normal Groups of Order 2 with identity '{}':
      PS3_subalgebra_2: ['{}', '{0, 2}']
      PS3_subalgebra_3: ['{}', '{1, 2}']
      PS3_subalgebra_5: ['{}', '{2}']
      PS3_subalgebra_7: ['{}', '{1}']
      PS3_subalgebra_8: ['{}', '{0}']
      PS3_subalgebra_9: ['{}', '{0, 1}']
      PS3_subalgebra_11: ['{}', '{0, 1, 2}']



### Subalgebras of Semigroups, Etc.

Recall the Semigroup example from above:

In [83]:
>>> sg.about()


Semigroup: Example 1.4.1
Instance ID: 140321287435856
Description: See: Groupoids and Smarandache Groupoids by W. B. Vasantha Kandasamy
Order: 6
Elements: ['a', 'b', 'c', 'd', 'e', 'f']
Identity: None
Associative? Yes
Commutative? No
Has Inverses? No
Cayley Table (showing indices):
[[0, 3, 0, 3, 0, 3],
 [1, 4, 1, 4, 1, 4],
 [2, 5, 2, 5, 2, 5],
 [3, 0, 3, 0, 3, 0],
 [4, 1, 4, 1, 4, 1],
 [5, 2, 5, 2, 5, 2]]


It contains 4 unique subalgebras, up to isomorphism, 3 Semigroups and 1 Group:

In [84]:
>>> sg_proper_subs = sg.proper_subalgebras()

>>> partitions = partition_into_isomorphic_lists(sg_proper_subs)

>>> about_isomorphic_partitions(sg, partitions)


Subalgebras of <Semigroup:Example 1.4.1, ID:140321287435856>
  There are 4 unique subalgebras, up to isomorphisms, out of 10 total subalgebras
  as shown by the partitions below:

3 Semigroups of Order 4:
      Example 1.4.1_subalgebra_0: ['a', 'c', 'd', 'f']
      Example 1.4.1_subalgebra_7: ['b', 'c', 'e', 'f']
      Example 1.4.1_subalgebra_8: ['a', 'b', 'd', 'e']

3 Semigroups of Order 2:
      Example 1.4.1_subalgebra_1: ['a', 'e']
      Example 1.4.1_subalgebra_5: ['a', 'c']
      Example 1.4.1_subalgebra_6: ['c', 'e']

3 Commutative Groups of Order 2:
      Example 1.4.1_subalgebra_2: ['c', 'f'] with identity 'c'
      Example 1.4.1_subalgebra_3: ['b', 'e'] with identity 'e'
      Example 1.4.1_subalgebra_9: ['a', 'd'] with identity 'a'

1 Semigroup of Order 3:
      Example 1.4.1_subalgebra_4: ['a', 'c', 'e']



## Built-In Examples

``Examples`` is a convenience class for accessing some of the example algebras in the algebras directory.  To add or subtract algebras to its default list, see the file, 'examples.json', in the algebras directory.

In [85]:
>>> from finite_algebras import Examples

>>> ex = Examples(alg_dir)  # Requires path to directory containing algebras' JSON files

                           Example Algebras
----------------------------------------------------------------------
  15 example algebras are available.
  Use "get_example(INDEX)" to retrieve a specific example,
  where INDEX is the first number on each line below:
----------------------------------------------------------------------
0: A4 -- Alternating group on 4 letters (AKA Tetrahedral group)
1: D3 -- https://en.wikipedia.org/wiki/Dihedral_group_of_order_6
2: D4 -- Dihedral group on four vertices
3: Pinter29 -- Non-abelian group, p.29, 'A Book of Abstract Algebra' by Charles C. Pinter
4: RPS -- Rock, Paper, Scissors Magma
5: S3 -- Symmetric group on 3 letters
6: S3X -- Another version of the symmetric group on 3 letters
7: V4 -- Klein-4 group
8: Z4 -- Cyclic group of order 4
9: F4 -- Field with 4 elements (from Wikipedia)
10: mag_id -- Magma with Identity
11: Example 1.4.1 -- See: Groupoids and Smarandache Groupoids by W. B. Vasantha Kandasamy
12: Ex6 -- Example 6: http://www-group

In [86]:
>>> grp = ex.get_example(3)
>>> grp.about()


Group: Pinter29
Instance ID: 140321287770960
Description: Non-abelian group, p.29, 'A Book of Abstract Algebra' by Charles C. Pinter
Order: 6
Identity: I
Associative? Yes
Commutative? No
Elements:
   Index   Name   Inverse  Order
      0       I       I       1
      1       A       A       2
      2       B       D       3
      3       C       C       2
      4       D       B       3
      5       K       K       2
Cayley Table (showing indices):
[[0, 1, 2, 3, 4, 5],
 [1, 0, 3, 2, 5, 4],
 [2, 5, 4, 1, 0, 3],
 [3, 4, 5, 0, 1, 2],
 [4, 3, 0, 5, 2, 1],
 [5, 2, 1, 4, 3, 0]]


## Cayley Tables

Under normal usage, there should be no need to directly create Cayley Tables. This section, however, provides a brief glimse at the ``CayleyTable`` class.

All of the properties of a finite algebra can be determined from its Cayley Table, or in the case of this Python module, its ``CayleyTable``.  That functionality is passed through to the appropriate methods of the various algebras.  Below, is a demonstration of how **distributivity** between two binary operations can be determined using their Cayley Tables.

The two tables, below, were generated from the powerset of a 3 element set, where "addition" is **symmetric difference** and "multiplication" is **intersection**.  Recall, the order of the powerset is $2^n$, where $n$ is the size of the set.

The element names are simply the string representations of the sets in the powerset:

['{}', '{0}', '{1}', '{2}', '{0, 1}', '{0, 2}', '{1, 2}', '{0, 1, 2}']

And the tables, below, contain the positions (indices) of the 8 elements in the powerset:

In [87]:
>>> addtbl = [[0, 1, 2, 3, 4, 5, 6, 7],
              [1, 0, 4, 5, 2, 3, 7, 6],
              [2, 4, 0, 6, 1, 7, 3, 5],
              [3, 5, 6, 0, 7, 1, 2, 4],
              [4, 2, 1, 7, 0, 6, 5, 3],
              [5, 3, 7, 1, 6, 0, 4, 2],
              [6, 7, 3, 2, 5, 4, 0, 1],
              [7, 6, 5, 4, 3, 2, 1, 0]]

In [88]:
>>> multbl = [[0, 0, 0, 0, 0, 0, 0, 0],
              [0, 1, 0, 0, 1, 1, 0, 1],
              [0, 0, 2, 0, 2, 0, 2, 2],
              [0, 0, 0, 3, 0, 3, 3, 3],
              [0, 1, 2, 0, 4, 1, 2, 4],
              [0, 1, 0, 3, 1, 5, 3, 5],
              [0, 0, 2, 3, 2, 3, 6, 6],
              [0, 1, 2, 3, 4, 5, 6, 7]]

In [89]:
>>> from cayley_table import CayleyTable

In [90]:
>>> addct = CayleyTable(addtbl)
>>> addct.about(True)

  Order  Associative?  Commutative?  Left Id?  Right Id?  Identity?  Inverses?
-------------------------------------------------------------------------------------
     8        True         True            0         0          0       True


In [91]:
>>> mulct = CayleyTable(multbl)
>>> mulct.about(True)

  Order  Associative?  Commutative?  Left Id?  Right Id?  Identity?  Inverses?
-------------------------------------------------------------------------------------
     8        True         True            7         7          7      False


### Checking Tables for Distributivity

Multiplication distributes over addition.

In [92]:
>>> mulct.distributes_over(addct)

True

But, addition does not distribute over multiplication.

In [93]:
>>> addct.distributes_over(mulct)

False

## Resources

* <b>Book</b>: ["Visual Group Theory" by Nathan Carter](https://bookstore.ams.org/clrm-32)
* [Group Explorer](https://nathancarter.github.io/group-explorer/index.html) -- Visualization software for the abstract algebra classroom
* [Groupprops, The Group Properties Wiki (beta)](https://groupprops.subwiki.org/wiki/Main_Page)
* [GroupNames](https://people.maths.bris.ac.uk/~matyd/GroupNames/index.html) -- "A database, under construction, of names, extensions, properties and character tables of finite groups of small order."
* [GAP](https://www.gap-system.org/#:~:text=What%20is%20GAP%3F,data%20libraries%20of%20algebraic%20objects.) -- "Groups, Algorithms, Programming - a System for Computational Discrete Algebra"
* [Groups of small order](http://www.math.ucsd.edu/~atparris/small_groups.html): Compiled by John Pedersen, Dept of Mathematics, University of South Florida
* [List of small groups](https://en.wikipedia.org/wiki/List_of_small_groups): Finite groups of small order up to group isomorphism
* [Classification of Groups of Order n ≤ 8 (PDF)](http://www2.lawrence.edu/fast/corrys/Math300/8Groups.pdf)
* [Subgroups of Order 4 (PDF)](http://newton.uor.edu/facultyfolder/beery/abstract_algebra/08_SbgrpsOrder4.pdf)
* Klein four-group, V4
  * [Wikipedia](https://en.wikipedia.org/wiki/Klein_four-group)
  * [Group Explorer](https://github.com/nathancarter/group-explorer/blob/master/groups/V_4.group)
* Cyclic group
  * [Wikipedia](https://en.wikipedia.org/wiki/Cyclic_group)
  * [Z4, cyclic group of order 4](https://github.com/nathancarter/group-explorer/blob/master/groups/Z_4.group)
* Symmetric group
  * [Symmetric group on 3 letters](https://github.com/nathancarter/group-explorer/blob/master/groups/S_3.group). Another name for this group is <i>"Dihedral group on 3 vertices"</i>
* [Groupoids and Smarandache Groupoids](https://arxiv.org/ftp/math/papers/0304/0304490.pdf) by W. B. Vasantha Kandasamy
* ["Rings and Fields"](http://www-groups.mcs.st-andrews.ac.uk/~john/MT4517/index.html), John O'Connor & Edmund Robertson, School of Math. & Stat., Univ. of St Andrews, Scotland